In [23]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
'''
@File         :ModuleTest.ipynb
@Description  :
@Time         :2022/04/26 06:11:00
@Author       :Hedwig
@Version      :1.0
'''
import torch.nn as nn
import torch
from collections import OrderedDict
import torch.functional as F

In [3]:
# 创建简单的模型
class LogicNet(nn.Module):
    def __init__(self,inputdim,hiddendim,outputdim):
        super(LogicNet,self).__init__()
        self.Linear1 = nn.Linear(inputdim,hiddendim)# 创建全连接层
        self.add_module("Linear2",nn.Linear(hiddendim,outputdim))# 与上一行的创建方式无本质区别
        # 还可以考虑更高级的ModuleList方法
        self.criterion = nn.CrossEntropyLoss()
    
    def forward(self,x):
        x = self.Linear1(x)
        x = torch.tanh(x)
        x = self.Linear2(x)
        return x
    
    def predict(self,x):
        pred = torch.softmax(self.forward(x),dim=1)
        return torch.argmax(pred,dim=1)

    def getloss(self,x,y):
        y_pred = self.forward(x)
        loss = self.criterion(y_pred,y)
        return loss


In [6]:
# Module的children方法可以获取Module类实例的各层信息
model = LogicNet(inputdim=2,hiddendim=3,outputdim=2)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
# 获取层信息
for sub_module in model.children():
    print(sub_module)
# 获取层和名字信息
for name,module in model.named_children():
    print(name,'is:',module)
# 获取整个网络的结构信息
for module in model.modules():
    print(module)

Linear(in_features=2, out_features=3, bias=True)
Linear(in_features=3, out_features=2, bias=True)
CrossEntropyLoss()
Linear1 is: Linear(in_features=2, out_features=3, bias=True)
Linear2 is: Linear(in_features=3, out_features=2, bias=True)
criterion is: CrossEntropyLoss()
LogicNet(
  (Linear1): Linear(in_features=2, out_features=3, bias=True)
  (Linear2): Linear(in_features=3, out_features=2, bias=True)
  (criterion): CrossEntropyLoss()
)
Linear(in_features=2, out_features=3, bias=True)
Linear(in_features=3, out_features=2, bias=True)
CrossEntropyLoss()


In [8]:
# Parameter类是Variable子类，模型中加入不同层的时候就会按该层的定义在模型中添加相应的参数
# 这些参数都是可学习参数，以下方式是通过定义网络层以外的方式向模型添加参数
# 为模型添加参数采用register_parameter(name,param)的方式添加
# 这个方法把参数写进了网络中，跟随网络训练而训练
# 神经网络搭建的时候有时需要保存一个状态，这个状态不是模型的参数(如BN层的均值和方差)
# 这时候可以用register_buffer(name,tensor)为模型添加状态参数
# 这个函数返回的不是模型的权重参数，他只是一个临时变量
for param in model.parameters():# 用这个查看模型的参数
    print(type(param.data),param.size())
for name,param in model.named_parameters():
    print(type(param.data),param.size(),name)


<class 'torch.Tensor'> torch.Size([3, 2])
<class 'torch.Tensor'> torch.Size([3])
<class 'torch.Tensor'> torch.Size([2, 3])
<class 'torch.Tensor'> torch.Size([2])
<class 'torch.Tensor'> torch.Size([3, 2]) Linear1.weight
<class 'torch.Tensor'> torch.Size([3]) Linear1.bias
<class 'torch.Tensor'> torch.Size([2, 3]) Linear2.weight
<class 'torch.Tensor'> torch.Size([2]) Linear2.bias


In [9]:
# 模型定义时，属性中不限制变量的类型，也可以定义self.a=3诸如此类，
# 但是在模型移植的时候我们只会加载可训练的参数，这些可训练参数可以用state_dict()函数获取
# 它识别可学习参数是这样识别的：
# Module子类的属性如果被赋予了Module子类、Parameter类或者buffer参数作为值，那么它是可学习参数
class ModulePar(nn.Module):
    def __init__(self):
        super(ModulePar,self).__init__()
        # 作为Module子类ModulePar的属性Linear1被赋予了Module子类Linear作为值，它是可学习参数
        self.Linear1 = nn.Linear(1,2)
        # 作为Module子类ModulePar的属性tensor被赋予了一个不是以上内容的值，它不是可学习参数
        self.tensor = torch.rand([1])
        # 作为Module子类ModulePar的属性param被赋予了Parameter，它是可学习参数
        self.param = nn.Parameter(torch.randn([1]))
        # buffer参数，是可学习参数
        self.register_buffer("buffer",torch.randn([2,3]))
model = ModulePar()
for par in model.state_dict():
    print(par,":",model.state_dict()[par])

param : tensor([-0.0240])
buffer : tensor([[ 1.3605, -1.6510, -0.4707],
        [-0.3103,  0.8497,  1.0285]])
Linear1.weight : tensor([[0.6534],
        [0.5043]])
Linear1.bias : tensor([-0.8386,  0.4592])


In [12]:
# 搭建卷积神经网络也可以用Sequential，它在简单网络里书写更加简洁，但是不太容易搭复杂的网络
model = nn.Sequential(OrderedDict([
    ('conv1',nn.Conv2d(1,20,5)),
    ('relu1',nn.ReLU()),
    ('conv2',nn.Conv2d(20,64,5)),
    ('relu2',nn.ReLU())
]))# 这个书写的时候甚至不用写forward函数，因为默认forward就是按着搭建的顺序连接
print(model)
print(model.state_dict()['conv1.weight'])
print(model.state_dict()['conv1.bias'])

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)


tensor([-0.0844, -0.1884, -0.0736,  0.0814,  0.0067, -0.0674,  0.1926,  0.0472,
         0.0425,  0.0572,  0.1535,  0.1007, -0.1479,  0.1875,  0.1376,  0.0156,
        -0.0091, -0.1659, -0.1673, -0.0697])

In [16]:
# 激活层
# 激活层都有两种形式，一种是类形式，一种是函数形式
data = torch.randn(2,2)-0.5
# 类形式如下初始化
tanh = nn.Tanh()
output = tanh(data)
print(data)
print(output)
# 函数形式直接调用
print(torch.tanh(data))

tensor([[-1.4166,  0.2200],
        [-1.9129,  0.8044]])
tensor([[-0.8889,  0.2166],
        [-0.9573,  0.6665]])
tensor([[-0.8889,  0.2166],
        [-0.9573,  0.6665]])


In [ ]:
# L2正则化
# Adam优化器中有个weight_decay参数，当对损失函数使用L2正则化的时候，求导结果相对于不适用正则化，
# 权值w变化量相差了一个值，这个值为：学习率*正则化系数*权值，所以如果想使用正则化，在adam优化器
# 的weight_decay参数设置一个值就好，这个值就是正则化系数
# 推导可以查看：https://zhuanlan.zhihu.com/p/388415560

In [ ]:
# Dropout的实现同样包括类实现和函数实现,有Dropout、Dropout2D、Dropout3D三种实现
# torch.dropout
# Dropout的类实现不用设置training参数，它会自动识别是否是training状态

In [21]:
# BN层也有类和函数两种实现，BatchNorm1d、BatchNorm2d、BatchNorm3d
data = torch.randn(2,3,4,1)
print(data)
# 类实现如下，它不用设置training参数
obn = nn.BatchNorm2d(3,affine=True)# 参数为图片通道数
print(obn.weight)
print(obn.bias)
print(obn.eps)
output = obn(data)
print(output,output.size())

tensor([[[[ 1.1638],
          [-0.6488],
          [-0.3210],
          [-0.6156]],

         [[-0.9440],
          [-0.4823],
          [-1.7982],
          [ 0.1563]],

         [[ 0.2894],
          [ 0.1336],
          [-0.2402],
          [ 0.0348]]],


        [[[-0.1778],
          [-0.8765],
          [ 0.1455],
          [ 0.4631]],

         [[-1.0449],
          [-0.1946],
          [ 0.4151],
          [-1.4212]],

         [[-0.7934],
          [ 0.4222],
          [-0.7207],
          [ 0.7674]]]])
Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)
1e-05
tensor([[[[ 2.0119],
          [-0.8545],
          [-0.3363],
          [-0.8020]],

         [[-0.3874],
          [ 0.2519],
          [-1.5699],
          [ 1.1359]],

         [[ 0.5944],
          [ 0.2885],
          [-0.4454],
          [ 0.0947]]],


        [[[-0.1098],
          [-1.2146],
          [ 0.4015],
          [ 0.9039]],

   

In [ ]:
# 模型保存与加载
torch.save(model.state_dict(),'./model.pth')# 保存模型
# 模型载入，后面的map_location使得模型同时载入0号和1号卡，不常用
model.load_state_dict(torch.load('./model.pth',map_location={'cuda:1':'cuda:0'}))

# 多卡GPU中单卡训练，保存时最好用cpu方式存储，如果用上述方式存储，存下来的模型还会记录gpu卡号信息
# 后面加载的时候需要用到
model.cpu().state_dict()#单卡
model.module.cpu().state_dict()#多卡

# 如果不想存成cpu形式，那就加载的时候指派到指定的gpu
# GPU1加载到GPU0
torch.load('model.pth',map_location={'cuda:1':'cuda:0'})
# GPU加载到cpu
torch.load('model.pth',map_location=lambda storage,loc:storage)

In [43]:
# 损失函数
# 以下是L1损失的类实现的使用，MSELoss、CrossEntropyLoss也是类似
# BCELoss是二分类，BCEWithLogitsLoss也是二分类损失，sigmoid+BCELoss，最后一层不使用Sigmoid时用这个函数
# CrossEntropyLoss多分类损失，包括了求softmax的操作
criterion = torch.nn.L1Loss()

In [22]:
# 优化器
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
# 查看优化器参数
list(optimizer.param_groups[0].keys())

['params', 'lr', 'betas', 'eps', 'weight_decay', 'amsgrad']

In [ ]:
# 学习率退化
# 定长阶跃退化
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=50,gamma=0.99)# 每50步*0.99
# 不定长阶跃,如下表示在200 700 800时*0.9
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[200,700,800],gamma=0.9)
# ReduceLROnPlateau退化

In [26]:
# 添加钩子函数
# 钩子函数定义方式：register_forward_hook(hook)
# 每次调用forward方法，这个hook都会被调用一次
# hook函数定义为hook(module,input,output),
# hook函数不能修改input跟output，它返回一个句柄handle，调用handle的remove方法可以将hook移除
def for_hook(module,input,output):
    print("模型：",module)
    for val in input:
        print('输入：',input)
    for out_val in output:
        print('输出：',out_val)
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
    def forward(self,x):
        return x+1
model = Model()
x = torch.randn(1,requires_grad=True)
handle = model.register_forward_hook(for_hook)
print('模型结果：',model(x))
# register_backward_hook()

模型： Model()
输入： (tensor([0.4162], requires_grad=True),)
输出： tensor(1.4162, grad_fn=<UnbindBackward0>)
模型结果： tensor([1.4162], grad_fn=<AddBackward0>)


In [ ]:
# optimizer.zero_grad方法一般每次迭代都要执行一次，硬件资源紧张的时候可以多次计算backward梯度以后
# 执行一次优化器step然后zero_grad
# 多任务场景下，分别各自的loss都backward完再相加

In [42]:
# 卷积层的实现也包括函数实现和类实现
#torch.nn.Conv2d()
#torch.nn.functional.conv2d(inchannel,outchannel,kernel_size)
# 一般卷积核参数都是随机初始化，如果想人为赋值，需要用类实现，如下
condv = torch.nn.Conv2d(1,2,kernel_size=1,padding=1,bias=False)
print(condv.weight)#(1,1,1,1)NCHW
condv.weight = torch.nn.Parameter(torch.ones([2,1,1,1]))# 如果尺寸不一致则会覆盖 不会报错
condv.weight

Parameter containing:
tensor([[[[-0.6788]]],


        [[[ 0.6917]]]], requires_grad=True)


Parameter containing:
tensor([[[[1.]]],


        [[[1.]]]], requires_grad=True)

In [ ]:
# 池化层也有类实现和函数实现
# 函数实现torch.nn.functional.max_pool1d、2d、3d或者avg

In [ ]:
# 多卡训练
device_count = torch.cuda.device_count()
print('cuda.device_count',device_count)
device_ids = list(range(device_count))
model = nn.DataParallel(model,device_ids=device_ids)
criterion = nn.DataParallel(criterion,device_ids=device_ids)
# 如果loss计算很消耗资源，建议将其放在forward方法中
# 优化器不需要并行，因为参数会同步覆盖
# 多卡保存模型代码
torch.save(model.module.state_dict(),'model.pth')
# 如果代码结束显存还占用，执行nvidia-smi -pm 1
# 实时查看内存
# pip install gpustat
# watch --color -n1 gpustat -cpu


In [45]:
# 实现LSTM和GRU
# torch.nn.LSTM
# 输出结果形状为(序列长度，批次个数，方向*隐藏节点个数)
# 隐藏层状态h、单元状态C形状为(方向*层数，批次个数，隐藏层节点个数)
# torch.nn.GRU
# 分布式采样接口
data = torch.rand(2,4)#传入形状为(batch size,num class)的样本
print(data)
torch.multinomial(data,1)# 按numclass的分布取指定个数的样本

tensor([[0.0014, 0.5008, 0.3068, 0.3104],
        [0.9461, 0.3268, 0.9035, 0.9273]])


tensor([[1],
        [2]])